# Aim

Get FFAST working at a basic level (no noise, no window_var, etc.).

- for reasonably high-snr like around 5db (without the bin-length gain) look at kay vs. kay2 my guess is kay will fail but kay2 will keep working.  how much db can you go lower with kay2. is kay2 alway better?
- how does all this compare to "new" method?
- keep an eye on the total number of samples used by each method.

In [1]:
%load_ext autoreload
%autoreload 2

from ffast_python import *
from matplotlib import pyplot as plt
import numpy as np
import time

from tqdm import tqdm_notebook as tqdm

In [2]:
params = make_args()

params.primes = [2, 3]
params.prime_powers = [7, 5]

params.bins = [i**j for i,j in zip(params.primes, params.prime_powers)]
print(params.bins)

params.length = np.prod(params.bins)
print(np.log2(params.length))

params.iterations = 50
params.snr = 5 - 20*np.log10(max(params.bins))
# params.snr = -26

params.sparsity = 1

# for ml
# params.bin_processing_method = 'ml'
# params.chains = 500
# params.delays = 1

# for kay
# params.bin_processing_method = 'kay'
# params.bin_processing_method = 'kay2'
# params.chains = 14 # needs to be int(np.floor(np.log2(params.length))) [rounds down]
# params.delays = 9 # parameter to play with

# for new
params.bin_processing_method = 'new'
params.chains = 1 # this is set to 1 always
params.delays = 8 # parameter to play with



config = Config(params)
config.compute_params()

input_signal = ExperimentInputSignal(config)
input_signal.process()
output_signal = ExperimentOutputSignal(config, input_signal)

[128, 243]
14.92481250360578


In [3]:
config.get_noise_sd()**2

398.1071705534973

In [4]:
# vars(config)

In [5]:
config.display()

Running experiment mode (for now)
Signal length: 31104
Signal sparsity: 1
Signal-to-noise ratio (dB): -26
Random phase
Delays: 8
Bins: 128 243
Bin processor: new


In [6]:
# plt.plot(np.real(input_signal.time_signal))
# plt.plot(np.imag(input_signal.time_signal))
# plt.show()

In [7]:
# len(input_signal.time_signal)

In [8]:
# signal_fft = np.fft.fft(input_signal.time_signal)
# plt.plot(np.abs(signal_fft))
# plt.title('loc: {}'.format(input_signal.freqs))
# plt.show()

In [9]:
# input_signal.time_signal *= 2 # demonstrating that the input signal can be changed

In [10]:
ffast = FFAST(config, input_signal, output_signal)

In [11]:
ffast.frontend.process()
print(ffast.frontend.get_used_samples_nb())

21476


In [12]:
params.bins

[128, 243]

In [13]:
ffast = FFAST(config, input_signal, output_signal)

if not config.help_displayed:
    # this disregards the time to make the signal
    time_initial = int(np.round(time.time() * 1000)) 

    iterations = config.iterations
    
    for i in tqdm(range(iterations)):
        input_signal.process() # ffast.get_delays() in the CPP code as an argument
        print('input freq: {}'.format(input_signal.freqs))
        
        print('true singleton bins:')
        for stage_idx, bin_size in enumerate(params.bins):
            print('    stage {} bin {}'.format(stage_idx, input_signal.freqs[0] % bin_size))
        
        ffast.process()
        output_signal.process()
        print('---')

    time_final = int(np.round(time.time() * 1000))

    ffast.display_results(time_final - time_initial)

input freq: [26571]
true singleton bins:
    stage 0 bin 75
    stage 1 bin 84
found a singleton at 20356 -- stage 0 -- bin 4
---
input freq: [27531]
true singleton bins:
    stage 0 bin 11
    stage 1 bin 72
found a singleton at 8965 -- stage 0 -- bin 5
---
input freq: [20524]
true singleton bins:
    stage 0 bin 44
    stage 1 bin 112
found a singleton at 9609 -- stage 0 -- bin 9
---
input freq: [14809]
true singleton bins:
    stage 0 bin 89
    stage 1 bin 229
found a singleton at 18820 -- stage 0 -- bin 4
---
input freq: [30050]
true singleton bins:
    stage 0 bin 98
    stage 1 bin 161
found a singleton at 30602 -- stage 0 -- bin 10
---



KeyboardInterrupt: 

In [ ]:
output_signal.statistics()[0]/params.iterations

In [ ]:
asdfasdf

# this part is for debugging

In [14]:
vars(config)

{'SNR_dB': -26,
 'apply_window_var': False,
 'bin_offsets': array([  0, 128]),
 'bin_processing_method': 'new',
 'bins': array([128, 243]),
 'bins_sum': 371,
 'chains_nb': 1,
 'count_samples': True,
 'default_delays': False,
 'delays_nb': 8,
 'delays_per_bunch_nb': 8,
 'display_iteration_time': False,
 'distribution': array([0., 1.]),
 'eff_snr': 0.0025118864315095794,
 'experiment_mode': False,
 'help_displayed': False,
 'input_file': '/Users/ocal/Library/Jupyter/runtime/kernel-5db4ada3-fa5b-43cd-ae64-d7daedab3b55.json',
 'iterations': 50,
 'length_factor': 1,
 'max_SNR_dB': -26,
 'maximum_likelihood': False,
 'min_fourier_magnitude': 1,
 'noisy': True,
 'off_grid': False,
 'off_grid_SNR_dB': 100,
 'output_file': None,
 'phases_nb': 0,
 'prime_powers': array([7, 5]),
 'primes': array([2, 3]),
 'quantization_bits_nb': 0,
 'quantize': False,
 'reconstruct_signal_in_backend': False,
 'signal_length': 31104,
 'signal_length_original': 31104,
 'signal_sparsity': 1,
 'signal_sparsity_peelin

In [15]:
input_signal.process() # ffast.get_delays() in the CPP code as an argument
print("true frequencies: {}".format(input_signal.freqs))

ffast.frontend.process()
# ffast.backend.process()
# output_signal.process()

true frequencies: [4569]


In [16]:
# plt.stem(np.linalg.norm(ffast.frontend.observation_matrix, axis=0)**2/ffast.config.delays_nb)
# plt.xlabel('bin absolute index')
# plt.show()

In [17]:
singleton_bin = input_signal.freqs[0] % params.bins[0]
print(singleton_bin)

89


In [18]:
binprocessor = BinProcessor(config, ffast.frontend.delays, ffast.frontend.observation_matrix)

In [19]:
# plt.stem(np.linalg.norm(ffast.frontend.observation_matrix, axis=0)**2/ffast.config.delays_nb)
# plt.axhline(binprocessor.thresholds[0])
# plt.xlabel('bin absolute index')
# plt.show()

In [20]:
binprocessor.adjust_to(singleton_bin, singleton_bin, 0)

In [21]:
binprocessor.is_zeroton()

energy: 450.55424352124834
threshold: 1592.4286822140893


True

In [22]:
binprocessor.process()

In [23]:
# binprocessor.find_location_new()
print(binprocessor.location)
print(input_signal.freqs)

4569
[4569]


In [ ]:
binprocessor.is_singleton()

In [ ]:
binprocessor.location

In [ ]:
binprocessor.amplitude

In [ ]:
binprocessor.noise

In [ ]:
binprocessor.thresholds[0]

In [ ]:
binprocessor.minimum_energy

In [ ]:
ffast.backend.decoded_frequencies

In [ ]:
output_signal.check_full_recovery()

In [ ]:
output_signal.statistics()

In [ ]:
ffast = FFAST(config, input_signal, output_signal)

In [ ]:
input_signal.process() # ffast.get_delays() in the CPP code as an argument
print(input_signal.freqs)

In [ ]:
ffast.process()

In [ ]:
output_signal.process()

In [ ]:
output_signal.statistics()